## European Soccer Analysis

Who is the best player in Europe?  That's a tough question to answer.  If we just think about who scores the most goals, we might be weighting an impact cateogry that actually doesn't lead to team success.  Perhaps a defender, helping keep clean sheets, is in fact a "better" player than a striker who runs up a goal tally against weaker opponents.  So how could we measure this?

One way would be to consider something like the "Wins Above Replacement" [WAR](https://www.mlb.com/glossary/advanced-stats/wins-above-replacement).  WAR measures a player's value in all facets of the game by deciphering how many more wins he's worth than a replacement-level player at his same position.  Another way to think about this would be to look at the average number of points a team earns (3 for a win, 1 for a draw, 0 for a loss) when a player is on the field vs. when they aren't.  For a single player, we can calculate these averages and determine if there is a statistically significant difference in these measures.  That's what we'll do here.  We'll start with the case of Lionel Messi, widely considered the best player of all time.  We can also run this analysis for all players in Europe, to whose team's on-field points average is most significnatly above their team's points average when they are not on the field.

I'll be using the European Soccer Database sourced from [Kaggle/Github](https://www.kaggle.com/datasets/hugomathien/soccer).  This data is stored ina  SQLite database, so the first steps in our analysis will involve connecting to the database, runnign queries on the included tables, and creating Pandas dataframes from the data.

In [1]:
# Import necessary libraries
import sqlite3
import pandas as pd
import numpy as np
import scipy
import re

In [2]:
# Connect to the database and create a cursor to query/parse the data
conn = sqlite3.connect('Assets/database.sqlite')
curs = conn.cursor()

We've established the connection and created the cursor, and now let's check what the tables names are. 

In [3]:
table_query = """SELECT name FROM SQLITE_SCHEMA
WHERE type='table'
ORDER BY name;"""
tables = [table[0] for table in curs.execute(table_query).fetchall()]
tables

['Country',
 'League',
 'Match',
 'Player',
 'Player_Attributes',
 'Team',
 'Team_Attributes',
 'sqlite_sequence']

Now we can see what type of information is included in each table.  We'll cycle through the tables and pull out the columns using the .description method.

In [4]:
table_columns = {}
for table in tables:
    table_columns[table] = [col[0] for col in curs.execute(f'SELECT * FROM {table}').description]

Next we'll create  dictionary of dataframes for each table.  We won't work with all the tables, but we'll have them at our disposal.

In [5]:
dfs = {}
for table in tables:
    query = f'SELECT * FROM {table};'
    table_table = curs.execute(query).fetchall()
    dfs[table] = pd.DataFrame(table_table, columns=table_columns[table])

Having pulled out all the data and stored it locally-- it's a lot, but not too much-- we'll close our connection and release the db.

In [6]:
conn.close()

Let's check how many instances are in each dataframe, and list out the number of features in each as well.

In [7]:
for idx in range(len(tables)):
    print(tables[idx],dfs[tables[idx]].shape)

Country (11, 2)
League (11, 3)
Match (25979, 115)
Player (11060, 7)
Player_Attributes (183978, 42)
Team (299, 5)
Team_Attributes (1458, 25)
sqlite_sequence (7, 2)


We're going to look at all games from any country and any league, so parsing those tables will be unecessary.  We will need information from the Match, Player, and Team tables, so let's check on the type of data included in each.

In [8]:
matches_df = dfs['Match']

In [9]:
matches_df.dtypes.value_counts()

float64    96
object     10
int64       9
dtype: int64

We've got mostly numeric values, but there are 10 objects/strings.  Let's see which features those are:

In [10]:
cols_objs = matches_df.select_dtypes('object').columns
cols_objs

Index(['season', 'date', 'goal', 'shoton', 'shotoff', 'foulcommit', 'card',
       'cross', 'corner', 'possession'],
      dtype='object')

Interesting that goal, shoton, etc. are not numeric. Let's look at one exampel entry:

In [11]:
matches_df.loc[:,cols_objs]['goal'].value_counts().index[1]

'<goal><value><comment>n</comment><stats><goals>1</goals><shoton>1</shoton></stats><event_incident_typefk>71</event_incident_typefk><elapsed>28</elapsed><player1>79253</player1><sortorder>0</sortorder><team>8529</team><id>3227996</id><n>19</n><type>goal</type><goal_type>n</goal_type></value><value><comment>n</comment><stats><goals>1</goals><shoton>1</shoton></stats><event_incident_typefk>71</event_incident_typefk><elapsed>87</elapsed><player1>108808</player1><sortorder>0</sortorder><team>8564</team><id>3228148</id><n>22</n><type>goal</type><goal_type>n</goal_type></value><value><comment>n</comment><stats><goals>1</goals><shoton>1</shoton></stats><event_incident_typefk>71</event_incident_typefk><elapsed>89</elapsed><player1>24549</player1><sortorder>0</sortorder><team>8564</team><id>3228153</id><n>24</n><type>goal</type><goal_type>n</goal_type></value></goal>'

Ok, so some of this information is either not inputted very well or won't be easy to work with.  Thinking about this more broadly, what we want to know is essentially the score for each team and who started the game.  We don't have substitute information readily available, so we'll have to confine our analysis slightly.  **We are going to look at only if a player started the match as an indiation of if they were on the field.**  From the object info then, we'll only care about the season and date information.  We aren't likely to use the date info, but season will be important.

In [12]:
cols_keep = []
cols_objs_keep = ['season','date']
cols_keep.extend(cols_objs_keep)

Now let's check a few of the numeric features, starting with integers:

In [13]:
cols_ints = matches_df.select_dtypes('int64').columns
cols_ints

Index(['id', 'country_id', 'league_id', 'stage', 'match_api_id',
       'home_team_api_id', 'away_team_api_id', 'home_team_goal',
       'away_team_goal'],
      dtype='object')

In [14]:
matches_df.loc[:,cols_ints].head()

,id,country_id,league_id,stage,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal
0,1,1,1,1,492473,9987,9993,1,1
1,2,1,1,1,492474,10000,9994,0,0
2,3,1,1,1,492475,9984,8635,0,3
3,4,1,1,1,492476,9991,9998,5,0
4,5,1,1,1,492477,7947,9985,1,3


We care about the teams playing and the score, we we'll keep categories associated with that info (`home_team_api_id`, `away_team_api_id`, `home_team_goal`,`away_team_goal`).  We'll also keep the `match_api_id` as a way to index the matches and sort them later.

None are missing so we can cull our category list.

In [15]:
cols_ints_keep = list(cols_ints)
cols_ints_keep.pop(0)
cols_ints_keep.pop(0)
cols_ints_keep.pop(0)
cols_ints_keep.pop(0)
cols_keep.extend(cols_ints_keep)

In [16]:
cols_ints_keep

['match_api_id',
 'home_team_api_id',
 'away_team_api_id',
 'home_team_goal',
 'away_team_goal']

Alright, onto floats:

In [17]:
cols_flts = matches_df.select_dtypes('float').columns
cols_flts

Index(['home_player_X1', 'home_player_X2', 'home_player_X3', 'home_player_X4',
       'home_player_X5', 'home_player_X6', 'home_player_X7', 'home_player_X8',
       'home_player_X9', 'home_player_X10', 'home_player_X11',
       'away_player_X1', 'away_player_X2', 'away_player_X3', 'away_player_X4',
       'away_player_X5', 'away_player_X6', 'away_player_X7', 'away_player_X8',
       'away_player_X9', 'away_player_X10', 'away_player_X11',
       'home_player_Y1', 'home_player_Y2', 'home_player_Y3', 'home_player_Y4',
       'home_player_Y5', 'home_player_Y6', 'home_player_Y7', 'home_player_Y8',
       'home_player_Y9', 'home_player_Y10', 'home_player_Y11',
       'away_player_Y1', 'away_player_Y2', 'away_player_Y3', 'away_player_Y4',
       'away_player_Y5', 'away_player_Y6', 'away_player_Y7', 'away_player_Y8',
       'away_player_Y9', 'away_player_Y10', 'away_player_Y11', 'home_player_1',
       'home_player_2', 'home_player_3', 'home_player_4', 'home_player_5',
       'home_player_6', 

We want to know who is playing, and that appears to be what we have in these categories.  We also have odds from the oddsmakers, which we don't care about for this analyssis.  So, let's keep all the columns that mention player.  But, we'll leave out the X and Y referenced players since that apparently identifies their position on the field, which again is not part of this analysis.

In [18]:
cols_flts_keep = []
for col in cols_flts:
    regexp = re.compile(r'.+player_[^XY]+')
    if regexp.search(col):
        cols_flts_keep.append(col)    

In [19]:
cols_keep.extend(cols_flts_keep)

From the categories we wanted to keep, we'll reduce our Match dataframe to only those categories.

In [20]:
matches_df = matches_df.loc[:,cols_keep]
print(matches_df.columns)
matches_df.set_index('match_api_id', inplace=True)

Index(['season', 'date', 'match_api_id', 'home_team_api_id',
       'away_team_api_id', 'home_team_goal', 'away_team_goal', 'home_player_1',
       'home_player_2', 'home_player_3', 'home_player_4', 'home_player_5',
       'home_player_6', 'home_player_7', 'home_player_8', 'home_player_9',
       'home_player_10', 'home_player_11', 'away_player_1', 'away_player_2',
       'away_player_3', 'away_player_4', 'away_player_5', 'away_player_6',
       'away_player_7', 'away_player_8', 'away_player_9', 'away_player_10',
       'away_player_11'],
      dtype='object')


Let's now drop out any instances that have missing data, and we'll see how many matches we lose due to missing data.

In [21]:
before_no_matches = matches_df.shape[0]
print('Number of matches in full dataset:',before_no_matches)
matches_df.dropna(inplace=True)
after_no_matches = matches_df.shape[0]
print('Number of matches with full starting player info:',after_no_matches)
lost_no_matches = before_no_matches - after_no_matches
print(f'We lost {lost_no_matches}, ({round(lost_no_matches/before_no_matches*100,1)}%)')

Number of matches in full dataset: 25979
Number of matches with full starting player info: 21374
We lost 4605, (17.7%)


We lost nearly 18%, but we still have a very large number of instances/matches to work with-- more than 21,300.

### Question 1: Is Lionel Messi's team significantly more likely to win when we starts for his team?

We'll focus on one player to start.  Lionel Messi is a superstar in Europe, winning every major trophy possible for club and country at this point.  He's won multiple Ballon'dOros and is arguably the greatest player in history.  But how much better is his club when he starts when he doesn't?  Let's work a bit with our dataframes for Players and Teams to make sure the data makes sense, and then we'll start to pull out statistics.

In [22]:
players_df = dfs['Player']
teams_df = dfs['Team']

In [23]:
teams_df.head()

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,1,9987,673.0,KRC Genk,GEN
1,2,9993,675.0,Beerschot AC,BAC
2,3,10000,15005.0,SV Zulte-Waregem,ZUL
3,4,9994,2007.0,Sporting Lokeren,LOK
4,5,9984,1750.0,KSV Cercle Brugge,CEB


We have two team id's-- one is for FIFA and one is unidentified.  Reading the docs, it seems like `team_api_id` is the one used by the match dataset to identify the teams playing.  But we can verify by pulling out the unique ids for the two team api options and seeing if any of those are missing from the match dataset.

In [24]:
hts = [idx[0] for idx in list(matches_df['home_team_api_id'].to_frame().values)]
team_api = list(teams_df['team_api_id'].values)
team_fifa = list(teams_df['team_fifa_api_id'].values)
api_count = 0
fifa_count = 0
for ht in hts:
    if ht not in team_api:
        api_count += 1
    if ht not in team_fifa:
        fifa_count += 1
print('Number of home_team_api_ids in matches without matching id in player_api_id:',api_count)
print('Number of home_team_api_ids in matches without matching id in player_fifa_api_id:',fifa_count)

Number of home_team_api_ids in matches without matching id in player_api_id: 0
Number of home_team_api_ids in matches without matching id in player_fifa_api_id: 21374


As suspected, the FIFA api id is misisng from (all) the match dataset, so the team api that connects to the match dataset (`home_team_api_id` and `away_team_api_id`) is `team_api_id`.

In [25]:
teams_cols_keep = ['team_api_id','team_long_name']
teams_df = teams_df.loc[:,teams_cols_keep]
teams_df.set_index('team_api_id',inplace=True)

In [26]:
teams_df.head()

,team_long_name
team_api_id,
9987,KRC Genk
9993,Beerschot AC
10000,SV Zulte-Waregem
9994,Sporting Lokeren
9984,KSV Cercle Brugge


We'll make the same check for players, as there are two api id's in the player dataset.  We'll check both the away and home players.

In [27]:
players_df.head()

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
2,3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
3,4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
4,5,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154


In [28]:
hp1 = [idx[0] for idx in list(matches_df['home_player_1'].to_frame().values)]
player_api = list(players_df['player_api_id'].values)
player_fifa = list(players_df['player_fifa_api_id'].values)
api_count = 0
fifa_count = 0
for hp in hp1[:1000]:
    if hp not in player_api:
        api_count += 1
    if hp not in player_fifa:
        fifa_count += 1
print('Number of home_player_1s in matches without matching id in player_api_id:',api_count)
print('Number of home_player_1s in matches without matching id in player_fifa_api_id:',fifa_count)

Number of home_player_1s in matches without matching id in player_api_id: 0
Number of home_player_1s in matches without matching id in player_fifa_api_id: 945


In [29]:
ap1 = [idx[0] for idx in list(matches_df['away_player_1'].to_frame().values)]
player_api = list(players_df['player_api_id'].values)
player_fifa = list(players_df['player_fifa_api_id'].values)
api_count = 0
fifa_count = 0
for ap in ap1[:1000]:
    if ap not in player_api:
        api_count += 1
    if ap not in player_fifa:
        fifa_count += 1
print('Number of away_player_1s in matches without matching id in player_api_id:',api_count)
print('Number of away_player_1s in matches without matching id in player_fifa_api_id:',fifa_count)

Number of away_player_1s in matches without matching id in player_api_id: 0
Number of away_player_1s in matches without matching id in player_fifa_api_id: 946


In both cases, we are NOT going to use the FIFA api for player id as it is the only one with missing id's from the match dataset.

In [30]:
players_cols_keep = ['player_api_id','player_name']
players_df = players_df.loc[:,players_cols_keep]
players_df.set_index('player_api_id', inplace=True)

In [31]:
players_df.head()

,player_name
player_api_id,
505942,Aaron Appindangoye
155782,Aaron Cresswell
162549,Aaron Doran
30572,Aaron Galindo
23780,Aaron Hughes


Let's focus on Messi now.  We'll first figure out what his player_id is:

In [32]:
messi_id_df = players_df[players_df['player_name'].str.findall(r'.+Messi').map(bool)]

In [33]:
messi_id = messi_id_df.index[0]
messi_id

30981

Now we can find how many of the matches he played in.

In [34]:
matches_df.apply(lambda x:x==messi_id).apply(any, axis=1).value_counts()

False    21149
True       225
dtype: int64

We know which matches he played in, but we also need to figure out which matches his team played when he was on the team and where he didn't play.  First, let's figure out the column index for the home and away players.

In [35]:
matches_df.columns

Index(['season', 'date', 'home_team_api_id', 'away_team_api_id',
       'home_team_goal', 'away_team_goal', 'home_player_1', 'home_player_2',
       'home_player_3', 'home_player_4', 'home_player_5', 'home_player_6',
       'home_player_7', 'home_player_8', 'home_player_9', 'home_player_10',
       'home_player_11', 'away_player_1', 'away_player_2', 'away_player_3',
       'away_player_4', 'away_player_5', 'away_player_6', 'away_player_7',
       'away_player_8', 'away_player_9', 'away_player_10', 'away_player_11'],
      dtype='object')

In [36]:
home_cols = []
for col in cols_flts:
    regexp = re.compile(r'.+player_[^XY]+')
    if regexp.search(col):
        cols_flts_keep.append(col)    

In [37]:
home_team_player_colidx = [i for i in range(6,17)]
home_team_player_colidx
matches_df.iloc[:,home_team_player_colidx].columns

Index(['home_player_1', 'home_player_2', 'home_player_3', 'home_player_4',
       'home_player_5', 'home_player_6', 'home_player_7', 'home_player_8',
       'home_player_9', 'home_player_10', 'home_player_11'],
      dtype='object')

In [38]:
away_team_player_colidx = [i for i in range(17,len(matches_df.columns))]
away_team_player_colidx
matches_df.iloc[:,away_team_player_colidx].columns

Index(['away_player_1', 'away_player_2', 'away_player_3', 'away_player_4',
       'away_player_5', 'away_player_6', 'away_player_7', 'away_player_8',
       'away_player_9', 'away_player_10', 'away_player_11'],
      dtype='object')

We can use the columns indexes to figure out which team Messi when playing on when he was a home team player.

In [39]:
messi_home_df = matches_df[matches_df.iloc[:,home_team_player_colidx].apply(lambda x:x==messi_id).apply(any, axis=1)]
messi_home_team = matches_df[matches_df.iloc[:,home_team_player_colidx]
                             .apply(lambda x:x==messi_id).apply(any, axis=1)]['home_team_api_id']
messi_home_team_id = messi_home_team.value_counts().index[0]
messi_home_team_id

8634

Likewise for when messi was an away player:

In [40]:
messi_away_df = matches_df[matches_df.iloc[:,away_team_player_colidx].apply(lambda x:x==messi_id).apply(any, axis=1)]
messi_away_team = matches_df[matches_df.iloc[:,away_team_player_colidx]
                             .apply(lambda x:x==messi_id).apply(any, axis=1)]['away_team_api_id']
messi_away_team_id = messi_away_team.value_counts().index[0]
messi_away_team_id

8634

Ok, so he's on the same team in both instances, as we expect.  I know he played for FC Barcelona, but let's now figure out the team that Messi was playing on in all of those matches.

In [41]:
messi_home_matches_idx = list(messi_home_df.index)
messi_away_matches_idx = list(messi_away_df.index)
messi_matches_idx = messi_home_matches_idx
messi_matches_idx.extend(messi_away_matches_idx)
messi_matches_idx = list(messi_matches_idx)

In [42]:
messi_team_id = messi_home_team_id
messi_team = teams_df.loc[messi_team_id].values[0]
messi_team

'FC Barcelona'

Good-- my expectation agrees with the data.  Let's also figure which seasons Messi played in.  I don't want to include seasons when Messi wasn't even in the team, so we'll only look at seasons when Messi played at least one game.

In [43]:
messi_seasons_ser = matches_df[matches_df.apply(lambda x:x==messi_id).apply(any, axis=1)]['season']

In [44]:
messi_seasons = list(set(messi_seasons_ser.values))
messi_seasons

['2010/2011',
 '2009/2010',
 '2008/2009',
 '2014/2015',
 '2012/2013',
 '2013/2014',
 '2015/2016',
 '2011/2012']

Ok, so let's figure out how many of the 225 games Messi played in where he was on the home team vs. the away team.  We'll need this to figure out if he was on the winning team or losing team.

In [45]:
(matches_df['home_team_api_id'].map(lambda x:x==messi_team_id)).value_counts()

False    21243
True       131
Name: home_team_api_id, dtype: int64

In [46]:
(matches_df['away_team_api_id'].map(lambda x:x==messi_team_id)).value_counts()

False    21235
True       139
Name: away_team_api_id, dtype: int64

Now let's determine how many matches in which FC Barcelona played within those seasons where Messi was on the team and active.

In [47]:
messi_team_matches_df = matches_df[(matches_df['season'].map(lambda x:x in messi_seasons)) &
           (
            (matches_df['home_team_api_id'].map(lambda x:x==messi_team_id)) | 
            (matches_df['away_team_api_id'].map(lambda x:x==messi_team_id))
           )]
print(messi_team_matches_df.shape)
messi_team_matches_df.head()

(270, 28)


,season,date,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_1,home_player_2,home_player_3,home_player_4,...,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11
match_api_id,,,,,,,,,,,,,,,,,,,,,
530240,2008/2009,2008-11-08 00:00:00,8634,10281,6,0,32657.0,33988.0,30740.0,37482.0,...,37597.0,37672.0,38065.0,2790.0,33586.0,33588.0,33820.0,75192.0,33729.0,39142.0
530329,2008/2009,2008-11-29 00:00:00,8302,8634,0,3,33986.0,70255.0,33812.0,33990.0,...,33988.0,30740.0,37482.0,30661.0,39854.0,36615.0,26146.0,30981.0,33639.0,30626.0
530337,2008/2009,2008-12-06 00:00:00,8634,10267,4,0,32657.0,33988.0,30740.0,30661.0,...,34007.0,34029.0,30994.0,34575.0,30666.0,38701.0,37824.0,32761.0,41468.0,30909.0
530343,2008/2009,2008-12-13 00:00:00,8634,8633,2,0,32657.0,33988.0,30740.0,30661.0,...,30658.0,30649.0,34520.0,30962.0,30878.0,32765.0,18814.0,42714.0,25759.0,30655.0
530358,2008/2009,2008-12-21 00:00:00,10205,8634,1,2,41466.0,33733.0,38458.0,56678.0,...,33988.0,37482.0,30738.0,30661.0,39854.0,154257.0,26146.0,30981.0,33639.0,30626.0


FC Barcelona played 270 matches in seasons where Messi played at least one game.  Messi played in 225 of those games.  Let's condense the dataframe slightly and build out a few features to make the calculations easier. 

In [48]:
cols_keep = list(messi_team_matches_df.columns)
cols_keep.pop(1)
for _ in range(len(cols_keep)-5):
    cols_keep.pop()
cols_keep

['season',
 'home_team_api_id',
 'away_team_api_id',
 'home_team_goal',
 'away_team_goal']

In [49]:
messi_team_matches_df = messi_team_matches_df.loc[:,cols_keep]
messi_team_matches_df.rename(columns={'home_team_api_id':'Home','away_team_api_id':'Away',
                                     'home_team_goal':'HomeScore','away_team_goal':'AwayScore'}, inplace=True)

Here's what are matches dataframe looks like, for games when Messi's team was playing:

In [50]:
messi_team_matches_df.head()

,season,Home,Away,HomeScore,AwayScore
match_api_id,,,,,
530240,2008/2009,8634,10281,6,0
530329,2008/2009,8302,8634,0,3
530337,2008/2009,8634,10267,4,0
530343,2008/2009,8634,8633,2,0
530358,2008/2009,10205,8634,1,2


Let's calculate the score for Messi's team, depending on if he is on the away side or the home side. We'll also then determine the Goal Differential (FC Barcelona - Opponent) to create a column that saves the result of the match, and we'll have one last category indicating if Messi started the game:

In [51]:
messi_team_matches_df['Barca'] = messi_team_matches_df['Home'].map(lambda x:'Home' if x == messi_team_id else 'Away')

In [52]:
messi_team_matches_df['Barca Score'] = messi_team_matches_df['HomeScore']
mask = messi_team_matches_df['Barca'] == 'Away'
messi_team_matches_df.loc[mask,'Barca Score'] = messi_team_matches_df.loc[mask,'AwayScore']

In [53]:
messi_team_matches_df['Opp Score'] = messi_team_matches_df['AwayScore']
mask = messi_team_matches_df['Barca'] == 'Away'
messi_team_matches_df.loc[mask,'Opp Score'] = messi_team_matches_df.loc[mask,'HomeScore']

In [54]:
messi_team_matches_df['GoalDiff'] = messi_team_matches_df['Barca Score'] - messi_team_matches_df['Opp Score']

In [55]:
messi_team_matches_df['Points'] = messi_team_matches_df['GoalDiff'].map(lambda x:
                                                                        3 if x>0 else 1 if x==0 else 0)
messi_team_matches_df['Result'] = messi_team_matches_df['GoalDiff'].map(lambda x:
                                                                        'Win' if x>0 else 'Draw' if x==0 else 'Loss')

In [56]:
messi_team_matches_df['Messi'] = False
messi_team_matches_df.loc[messi_matches_idx,'Messi'] = True

In [57]:
messi_team_matches_df.head()

,season,Home,Away,HomeScore,AwayScore,Barca,Barca Score,Opp Score,GoalDiff,Points,Result,Messi
match_api_id,,,,,,,,,,,,
530240,2008/2009,8634,10281,6,0,Home,6,0,6,3,Win,True
530329,2008/2009,8302,8634,0,3,Away,3,0,3,3,Win,True
530337,2008/2009,8634,10267,4,0,Home,4,0,4,3,Win,True
530343,2008/2009,8634,8633,2,0,Home,2,0,2,3,Win,True
530358,2008/2009,10205,8634,1,2,Away,2,1,1,3,Win,True


For our analysis, we'll be comparing two means for points.  When comparing two population means, we'll need to determine the standard error, which will involve knowing the sample sizes and the standard deviations (ddof=1) for each population.  We'll also need, to determine our two-sample t-test statistic, if the population variances are equal.  If they are, we can use a pooled method.  If not, we can use the unpooled (Welch's).  We will need to determine the df in this case, which is more difficult, but we can rely on statistical packages to assist. Let's first formalize our null and alternative hypotheses:

#### Null Hypothesis:
The average points earned per game by FC Barcelona in seasons in which Messi is active as a starting player and starts the game is higher than the average point earned per game by FC Barcelona in the same seasons but when Messi does not start.

**&mu;_starts > &mu;_sits**

The alternative hypothesis is that there is not difference in the averages noted above.

**&mu;_starts <= &mu;_sits**

We'll set our significance level to 5%:

&alpha; = 0.05

In [58]:
games_wo, games_with = list(messi_team_matches_df.groupby('Messi')['Points'].count().values)
print("FC Barcleona games played with Messi starting:",games_with)
print("FC Barcleona games played without Messi starting:",games_wo)
n1 = games_with
n2 = games_wo

FC Barcleona games played with Messi starting: 225
FC Barcleona games played without Messi starting: 45


In [59]:
points_ave_wo, points_ave_with = list(messi_team_matches_df.groupby('Messi')['Points'].mean().values)
print("FC Barcleona average points per game with Messi starting:",points_ave_with)
print("FC Barcleona average points per game without Messi starting:",points_ave_wo)
x1 = points_ave_with
x2 = points_ave_wo

FC Barcleona average points per game with Messi starting: 2.466666666666667
FC Barcleona average points per game without Messi starting: 2.4444444444444446


In [60]:
points_std_wo, points_std_with = list(messi_team_matches_df.groupby('Messi')['Points'].std(ddof=1).values)
print("FC Barcleona points standard deviation with Messi starting:",points_std_with)
print("FC Barcleona points standard deviation without Messi starting:",points_std_wo)
s1 = points_std_with
s2 = points_std_wo

FC Barcleona points standard deviation with Messi starting: 1.0264362759428514
FC Barcleona points standard deviation without Messi starting: 1.0125475422184012


The standard deviations in points is very similar between these two cases, so we can use the pooled method.  Let's calcualte the standard error.

In [61]:
sp = np.sqrt(
    ((n1 - 1) * s1 ** 2 + (n2 - 1) * s2 ** 2) / 
    (n1 + n2 - 2)
)
sp

1.0241689589050937

In [62]:
pooled_se = sp * np.sqrt(1 / n1 + 1 / n2)
pooled_se

0.16724609064766358

The degrees of freedom is much easier to calculate with the pooled method:

In [63]:
dof = n1 + n2 - 2
dof

268

In [64]:
diff_avepoints = x1 - x2
diff_avepoints

0.022222222222222143

Now we can use these stats to determine the t statistic based on the difference in sample means and standard error.

In [65]:
t_stat = diff_avepoints / pooled_se
t_stat

0.13287140007976375

Now we'll figure out the p-value and and compare it to alpha.  We are just concerned with one tail as our hypothesis was that FC Barcelona was better (not different) when Messi started vs. when he sat at the start of the game.

In [66]:
one_tailed_prob = 1 - scipy.stats.t.cdf(t_stat, dof)
"Our p-value is: {:.5e}".format(one_tailed_prob)

'Our p-value is: 4.47197e-01'

Our alpha was 0.05, but our p-value is much higher, 0.447.  **We have strong evidence, then, to reject the null hypothesis, that Messi makes FC Barcelona a significantly better team when he starts**.  This is somewhat suprrising, but we have to remember that FC Barcelona was one of the best teams in the world at this time, and he was surrounded by a ton of great players.  This investigation doesn't speak to Messi's value-- he may be making a great team even better-- but it also doesn't suggest that FC Barcelona would be a significantly weaker team without him.

### Question 2: Who are the most and least "valuable" players in Europe?

Well, again, we can't figure this out exactly.  Our investigation above doesn't rule out Messi being the most valuable person.  But let's at least look at who stands out statistically as most signficantly being associated wtih an increase in the average points their team earns per game when they start, and vice-versa.  We'll use the same steps as we did before, but we'll run through each player to spit out their statistics and then use that library to conduct the analysis.  To limit the number of players we'll run through, we'll only look at players that have played more than 30 games.  Later, we'll also make sure there are at least 30 games on record for the team where the player has not played.

In [76]:
player_ids = list(players_df.index)
player_ids_30plusgames = []
for player_id in player_ids:
    matches = matches_df.apply(lambda x:x==player_id).any(axis=1).sum()
    if matches >= 30:
        player_ids_30plusgames.append(player_id)

In [77]:
len(player_ids_30plusgames)

4947

We went from more than 11,000 players to now closer to 5,000, which is helpful, since we are using some heavy processing methods/functions.

In [78]:
player_results = {}
five_perc = int(len(player_ids_30plus)/20)

In [79]:
for i, player_id in enumerate(player_ids_30plus):
    if i>0 and i % five_perc == 0:
        print(f'Completed {i/five_perc*5}%')
    # Make dataframe with matches where the player is on the home team
    player_home_df = matches_df[matches_df.iloc[:,home_team_player_colidx].apply(lambda x:x==player_id).apply(any, axis=1)]
    # Identify the player's home team id
    player_home_team = matches_df[matches_df.iloc[:,home_team_player_colidx]
                             .apply(lambda x:x==player_id).apply(any, axis=1)]['home_team_api_id']
    player_home_team_ids = list(player_home_team.value_counts().index)
    # Make dataframe with matches where the player is on the away team
    player_away_df = matches_df[matches_df.iloc[:,away_team_player_colidx].apply(lambda x:x==player_id).apply(any, axis=1)]
    # Identify the player's away team id
    player_away_team = matches_df[matches_df.iloc[:,away_team_player_colidx]
                                 .apply(lambda x:x==player_id).apply(any, axis=1)]['away_team_api_id']
    player_away_team_ids = list(player_away_team.value_counts().index)
    # Don't include players that have more than one team
    if (len(player_home_team_ids) > 1) or (len(player_away_team_ids) > 1):
        continue
    if len(player_home_team_ids) == 1:
        player_home_team_id = player_home_team_ids[0]
    if len(player_away_team_ids) == 1:
        player_away_team_id = player_away_team_ids[0]
    # Don't include players that have only played away or only home games for their team
    if len(player_home_team_ids) == 0 or len(player_away_team_ids) == 0:
        continue
    # Identify the player's core team
    player_home_team = teams_df.loc[player_home_team_id].values[0]
    player_away_team = teams_df.loc[player_away_team_id].values[0]
    if player_home_team != player_away_team:
        continue
    player_team_id = player_home_team_id
    player_team = player_home_team
    # Find the indexes of the matches where the player is on the home or away team
    player_home_matches_idx = list(player_home_df.index)
    player_away_matches_idx = list(player_away_df.index)
    player_matches_idx = player_home_matches_idx
    player_matches_idx.extend(player_away_matches_idx)
    player_matches_idx = list(player_matches_idx)
    # Get the list of seasons where the player was playing for a team
    player_seasons_series = matches_df[matches_df.apply(lambda x:x==player_id).apply(any, axis=1)]['season']
    player_seasons = list(set(player_seasons_series.values))
    # Create a dataframe with all of the matches in which the player started games with their core team
    player_team_matches_df = matches_df[(matches_df['season'].map(lambda x:x in player_seasons)) &
           (
            (matches_df['home_team_api_id'].map(lambda x:x==player_team_id)) | 
            (matches_df['away_team_api_id'].map(lambda x:x==player_team_id))
           )]
    # Update dataframe with simplified columns
    player_team_matches_df = player_team_matches_df.loc[:,cols_keep]
    player_team_matches_df.rename(columns={'home_team_api_id':'Home','away_team_api_id':'Away',
                                     'home_team_goal':'HomeScore','away_team_goal':'AwayScore'}, inplace=True)
    # Add columns for Team status as home or away
    player_team_matches_df['Team'] = player_team_matches_df['Home'].map(
        lambda x:'Home' if x == player_team_id else 'Away')
    # Create columns for the Team's score, depending on if they are home or away
    player_team_matches_df['Team Score'] = player_team_matches_df['HomeScore']
    mask = player_team_matches_df['Team'] == 'Away'
    player_team_matches_df.loc[mask,'Team Score'] = player_team_matches_df.loc[mask,'AwayScore']
    # Do the same for the opponent's score
    player_team_matches_df['Opp Score'] = player_team_matches_df['AwayScore']
    mask = player_team_matches_df['Team'] == 'Away'
    player_team_matches_df.loc[mask,'Opp Score'] = player_team_matches_df.loc[mask,'HomeScore']
    # Create a category for GoalDiff, to use to determine result and points; create those columns after
    player_team_matches_df['GoalDiff'] = player_team_matches_df['Team Score'] - player_team_matches_df['Opp Score']
    player_team_matches_df['Points'] = player_team_matches_df['GoalDiff'].map(lambda x:
                                                                        3 if x>0 else 1 if x==0 else 0)
    player_team_matches_df['Result'] = player_team_matches_df['GoalDiff'].map(lambda x:
                                                                        'Win' if x>0 else 'Draw' if x==0 else 'Loss')
    # Create category for if the player started the match
    player_team_matches_df['player'] = False
    player_team_matches_df.loc[player_matches_idx,'player'] = True
    # Calculate the number of games played by the player and without the player starting; overlook players who never
    # one of their teams games
    games = list(player_team_matches_df.groupby('player')['Points'].count().values)
    if len(games) == 2:
        games_wo = games[0]
        games_with = games[1]
    else:
        continue
    # Do the same for the average number of points
    points_ave_wo, points_ave_with = list(player_team_matches_df.groupby('player')['Points'].mean().values)
    points_std_wo, points_std_with = list(player_team_matches_df.groupby('player')['Points'].std(ddof=1).values)
    try:
        games_win_wo = player_team_matches_df.groupby(['player','Result'])['Result'].count().to_frame(
        ).unstack().droplevel(0,axis=1).loc[False,'Win']
    except:
        games_win_wo = 0
    try:
        games_win_with = player_team_matches_df.groupby(['player','Result'])['Result'].count().to_frame(
        ).unstack().droplevel(0,axis=1).loc[True,'Win']
    except:
        games_win_with = 0
    player_results[player_id] = {'games_wo':games_wo,'games_with':games_with,
                                 'points_ave_wo':points_ave_wo,'points_ave_with':points_ave_with,
                                 'points_std_wo':points_std_wo,'points_std_with':points_std_with}

Completed 5.0%
Completed 10.0%
Completed 15.0%
Completed 20.0%
Completed 25.0%
Completed 30.0%
Completed 35.0%
Completed 40.0%
Completed 45.0%
Completed 50.0%
Completed 55.0%
Completed 60.0%
Completed 65.0%
Completed 70.0%
Completed 75.0%
Completed 80.0%
Completed 85.0%
Completed 90.0%
Completed 95.0%
Completed 100.0%


Done!  Finally.  Let's now limit this list to players that have played 30 games at least, and which hvae teams that have at least 30 games without them in the same seasons.

In [109]:
player_results_30 = {}
for player, stats in player_results.items():
    if stats['games_wo'] >= 30 and stats['games_with'] >= 30:
        player_results_30[player] = stats
len(player_results_30)

1049

We've got just over 1,000 players to consider now.  Let's calculate the p_values.  We will assume pooled t-tests is apporpriate.  Again, we need similar variance, and we need independent samples from something of a normal distribution.  We are at least close in al of these considerations.

In [110]:
p_values = {}
for player, stats in player_results_30.items():
    n1 = stats['games_with']
    n2 = stats['games_wo']
    x1 = stats['points_ave_with']
    x2 = stats['points_ave_wo']
    s1 = stats['points_std_with']
    s2 = stats['points_std_wo']  
    sp = np.sqrt(
    ((n1 - 1) * s1 ** 2 + (n2 - 1) * s2 ** 2) / 
    (n1 + n2 - 2)
    )
    pooled_se = sp * np.sqrt(1 / n1 + 1 / n2)
    dof = n1 + n2 - 2
    diff_avepoints = x1 - x2
    t_stat = diff_avepoints / pooled_se
    p_value = 1 - scipy.stats.t.cdf(t_stat, dof)
    p_values[player] = p_value

Now we can save the tope 10 and bottom 10 p_values-- the smallest p_values being the most significant points contributors and the highest being the most significant points detractors.

In [118]:
p_values_sorted = sorted(p_values.items(), key=lambda x:x[1])
top10 = p_values_sorted[:10]
bottom10 = p_values_sorted[-10:]

Let's print the results!  We'll list players, their teams, and their teams average poitns haul with and without the player in question.

In [152]:
print("Top Players in Europe (Sort of):\n")
for i, (player_id, p) in enumerate(top10):
    player_home_df = matches_df[matches_df.iloc[:,home_team_player_colidx].apply(lambda x:x==player_id).apply(any, axis=1)]
    player_home_team = matches_df[matches_df.iloc[:,home_team_player_colidx]
                             .apply(lambda x:x==player_id).apply(any, axis=1)]['home_team_api_id']
    team_id = list(player_home_team.value_counts().index)[0]
    team = teams_df.loc[team_id].values[0]
    print(f'{i+1}.',players_df.loc[player_id].values[0],'with',team,'\n',
          'Average Points With:',round(player_results[player_id]['points_ave_with'],3),
         '\n Average Points Without:',round(player_results[player_id]['points_ave_wo'],3),'\n')

Top Players in Europe (Sort of):

1. Leon Osman with Everton 
 Average Points With: 1.722 
 Average Points Without: 1.185 

2. Koke with Atlético Madrid 
 Average Points With: 2.17 
 Average Points Without: 1.532 

3. Antonio Di Natale with Udinese 
 Average Points With: 1.556 
 Average Points Without: 0.988 

4. Loic Perrin with AS Saint-Étienne 
 Average Points With: 1.661 
 Average Points Without: 1.142 

5. Lior Rafaelov with Club Brugge KV 
 Average Points With: 2.322 
 Average Points Without: 1.632 

6. Marco Schoenbaechler with FC Zürich 
 Average Points With: 1.728 
 Average Points Without: 1.198 

7. Paul Paton with Dundee United 
 Average Points With: 1.531 
 Average Points Without: 0.769 

8. Giorgos Galitsios with Sporting Lokeren 
 Average Points With: 1.765 
 Average Points Without: 1.062 

9. Michel Renggli with FC Luzern 
 Average Points With: 1.595 
 Average Points Without: 0.978 

10. Sam Nicholson with Heart of Midlothian 
 Average Points With: 1.718 
 Average Points

In [154]:
print("Bottom Players in Europe (Sort of):\n")
for i, (player_id, p) in enumerate(bottom10):
    player_home_df = matches_df[matches_df.iloc[:,home_team_player_colidx].apply(lambda x:x==player_id).apply(any, axis=1)]
    player_home_team = matches_df[matches_df.iloc[:,home_team_player_colidx]
                             .apply(lambda x:x==player_id).apply(any, axis=1)]['home_team_api_id']
    team_id = list(player_home_team.value_counts().index)[0]
    team = teams_df.loc[team_id].values[0]
    print(f'{i+1}.',players_df.loc[player_id].values[0],'with',team,'\n',
          'Average Points With:',round(player_results[player_id]['points_ave_with'],3),
          '\n Average Points Without:',round(player_results[player_id]['points_ave_wo'],3),'\n')

Bottom Players in Europe (Sort of):

1. Ferry de Regt with VVV-Venlo 
 Average Points With: 0.371 
 Average Points Without: 1.028 

2. David Meul with Willem II 
 Average Points With: 0.588 
 Average Points Without: 1.344 

3. Holger Badstuber with FC Bayern Munich 
 Average Points With: 2.098 
 Average Points Without: 2.511 

4. Craig Thomson with Heart of Midlothian 
 Average Points With: 1.062 
 Average Points Without: 1.912 

5. Alessandro Del Piero with Juventus 
 Average Points With: 1.426 
 Average Points Without: 1.987 

6. Fraser Kerr with Motherwell 
 Average Points With: 0.875 
 Average Points Without: 1.694 

7. Mario Gaspar with Villarreal CF 
 Average Points With: 1.336 
 Average Points Without: 1.94 

8. Oier with CA Osasuna 
 Average Points With: 0.578 
 Average Points Without: 1.19 

9. Ryan Esson with Inverness Caledonian Thistle 
 Average Points With: 1.08 
 Average Points Without: 1.611 

10. Garry Kenneth with Dundee United 
 Average Points With: 1.3 
 Average Poin

Any suprises?  I'm not surprised about Leon Osman-- he's a solid contribution on both sides of the ball.  Maybe doesn't score the most goals, but I would say subjectively he certainly made his team better.  On the other side, I'm surprised to see Del Piero, but he is on the older side, so maybe he's been getting more playing time than he should out of deference.